#### Байесовская классификация спам/не спам

Импортируем библиотеки

In [29]:
import pandas as pd
import sklearn.feature_extraction.text as tx, sklearn.naive_bayes as nb, sklearn.linear_model as lm, sklearn.ensemble as en, sklearn.metrics as me, sklearn.model_selection as ms
from warnings import filterwarnings as fw
fw('ignore')

Узнаем вариант

In [30]:
ord('А')%3

2

Прочитаем корпус текстов

In [31]:
corpus = pd.read_csv('Emailspam2.csv')
corpus

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


Проанализируем количество слов по классам:

In [32]:
cnt, cntS, cntH = 0, 0, 0
for i in corpus['text']:
    cnt += len(i.split(' '))
for i in corpus[corpus['label']=='spam']['text']:
    cntS += len(i.split(' '))
for i in corpus[corpus['label']=='ham']['text']:
    cntH += len(i.split(' '))
print('Всего строк:', corpus.shape[0], '\nИз них', 
    corpus['label'].value_counts()['spam'], 'spam и',corpus['label'].value_counts()['ham'], 'ham', 
    '\nВсего слов: ', cnt, '\nиз них', cntS, 'spam и', cntH, 'ham')

Всего строк: 5171 
Из них 1499 spam и 3672 ham 
Всего слов:  1083244 
из них 320863 spam и 762381 ham


In [33]:
xtrain, xtest, ytrain, ytest = ms.train_test_split(corpus['text'],corpus['label'], test_size=0.2)
print('Обучающая выборка:', xtrain.shape,'\nТестовая выборка', xtest.shape)

Обучающая выборка: (4136,) 
Тестовая выборка (1035,)


-- -- 
Используем все слова с учетом регистра:

In [34]:
cv = tx.CountVectorizer(lowercase=False)
cv.fit(corpus['text'])
print('Всего', len(cv.vocabulary_.keys()), 'токенов')

tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 50448 токенов


In [35]:
lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.99309   0.97823   0.98561       735
        spam    0.94855   0.98333   0.96563       300

    accuracy                        0.97971      1035
   macro avg    0.97082   0.98078   0.97562      1035
weighted avg    0.98018   0.97971   0.97982      1035
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.96929   0.98776   0.97844       735
        spam    0.96853   0.92333   0.94539       300

    accuracy                        0.96908      1035
   macro avg    0.96891   0.95554   0.96191      1035
weighted avg    0.96907   0.96908   0.96886      1035
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98909   0.98639   0.98774       735
        spam    0.96689   0.97333   0.97010       300

    accuracy                        0.98261      1035
   macro avg    0.97799   0.97986   0.97892      1035
weighted avg 

-- -- 
Используем все слова, но в одном регистре:

In [36]:
cv = tx.CountVectorizer()
cv.fit(corpus['text'])
print('Всего', len(cv.vocabulary_.keys()), 'токенов')

tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 50447 токенов


In [37]:
lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.99309   0.97823   0.98561       735
        spam    0.94855   0.98333   0.96563       300

    accuracy                        0.97971      1035
   macro avg    0.97082   0.98078   0.97562      1035
weighted avg    0.98018   0.97971   0.97982      1035
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97323   0.98912   0.98111       735
        spam    0.97222   0.93333   0.95238       300

    accuracy                        0.97295      1035
   macro avg    0.97272   0.96122   0.96674      1035
weighted avg    0.97294   0.97295   0.97278      1035
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98910   0.98776   0.98843       735
        spam    0.97010   0.97333   0.97171       300

    accuracy                        0.98357      1035
   macro avg    0.97960   0.98054   0.98007      1035
weighted avg 

-- -- 
Найдем все стоп-слова, которые встречаются в большом количестве сообщений (более 500 , что около 9% всей выборки) и уберем их:

In [38]:
cv = tx.CountVectorizer(max_df = 500)
cv.fit(corpus['text'])
stop = cv.stop_words_
print('Всего', len(stop), 'токенов')
print(stop)

cv = tx.CountVectorizer(analyzer = 'word', stop_words = stop)
cv.fit(corpus['text'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 117 токенов
{'would', 'this', 'from', 'message', 'some', 'here', 'by', 'to', 'forwarded', 'are', 'was', 'into', 'the', 'month', 'that', 'no', 'know', 'subject', '11', 'information', 'about', 'new', 'am', '03', 'enron', 'do', '000', 'you', 'mmbtu', '20', 'out', 'been', 'please', '02', 'and', 'get', 'http', 'or', 'be', 'have', 'farmer', '2000', 'corp', 'one', 'has', 'ect', 'can', 'it', 'my', 'see', 'of', 'price', 'up', 'in', '00', 'need', 'will', 'xls', 're', 'let', '10', 'just', 'attached', 'sent', '30', 'call', 'like', 'us', 'me', 'hpl', 'we', 'as', 'daren', 'time', 'if', 'gas', 'at', 'our', 'volume', 'is', 'on', 'not', 'day', 'more', '12', 'which', 'file', 'com', 'pm', '2001', 'email', 'your', 'nom', 'meter', 'thanks', 'they', 'for', 'only', 'all', 'there', 'what', 'should', 'any', 'an', 'hou', 'so', 'questions', 'cc', 'deal', '01', 'may', 'when', 'but', 'now', 'with', 'these', 'also'}


In [39]:
lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.97844   0.98776   0.98307       735
        spam    0.96928   0.94667   0.95784       300

    accuracy                        0.97585      1035
   macro avg    0.97386   0.96721   0.97046      1035
weighted avg    0.97578   0.97585   0.97576      1035
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.98476   0.96735   0.97598       735
        spam    0.92332   0.96333   0.94290       300

    accuracy                        0.96618      1035
   macro avg    0.95404   0.96534   0.95944      1035
weighted avg    0.96696   0.96618   0.96639      1035
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98372   0.98639   0.98505       735
        spam    0.96644   0.96000   0.96321       300

    accuracy                        0.97874      1035
   macro avg    0.97508   0.97320   0.97413      1035
weighted avg 

Найдем все стоп-слова, которые встречаются в маленьком количестве сообщений (менее 10, что около 0.18% всей выборки), отфильтруем только те слова, которые характерны какому-то классу (разница в частотах по классам более, чем в 7 раза):

In [40]:
cv = tx.CountVectorizer(min_df = 10)
cv.fit(corpus['text'])
stop = cv.stop_words_
print('Всего', len(stop), 'токенов отмечены как редкие.')
#print(stop) # не будем выводить список слов, так как он слишком большой

all_spam = corpus[corpus['label']=='spam'].groupby('label').agg({'text':' '.join}).to_string()
all_ham = corpus[corpus['label']=='ham'].groupby('label').agg({'text':' '.join}).to_string()
temp = set()
for i in stop:
    frH = all_ham.count(i + ' ')/cntH
    frS = all_spam.count(i + ' ')/cntS
    if(frS != 0 and frH != 0):
        if(not(frS/frH > 7 or frS/frH < 1/7)):
            temp.add(i)
stop = temp
print('Из них {0} удовлетворяют условиям.'.format(len(stop)))

cv = tx.CountVectorizer(analyzer = 'word', stop_words = stop)
cv.fit(corpus['text'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)
print('Работаем с {0} - {1} = {2}'.format(len(cv.vocabulary_) + len(stop), len(stop), len(cv.vocabulary_)), 'токенами.')

Всего 45591 токенов отмечены как редкие.


KeyboardInterrupt: 

In [ ]:
lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.98893   0.97147   0.98012       736
        spam    0.93269   0.97324   0.95254       299

    accuracy                        0.97198      1035
   macro avg    0.96081   0.97236   0.96633      1035
weighted avg    0.97269   0.97198   0.97215      1035
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.96925   0.98505   0.97709       736
        spam    0.96167   0.92308   0.94198       299

    accuracy                        0.96715      1035
   macro avg    0.96546   0.95407   0.95953      1035
weighted avg    0.96706   0.96715   0.96695      1035
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99186   0.99321   0.99253       736
        spam    0.98322   0.97993   0.98157       299

    accuracy                        0.98937      1035
   macro avg    0.98754   0.98657   0.98705      1035
weighted avg 

Используем встроенный список стоп-слов:

In [55]:
cv = tx.CountVectorizer()
cv.fit(corpus['text'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.99309   0.97823   0.98561       735
        spam    0.94855   0.98333   0.96563       300

    accuracy                        0.97971      1035
   macro avg    0.97082   0.98078   0.97562      1035
weighted avg    0.98018   0.97971   0.97982      1035
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97718   0.99048   0.98378       735
        spam    0.97586   0.94333   0.95932       300

    accuracy                        0.97681      1035
   macro avg    0.97652   0.96690   0.97155      1035
weighted avg    0.97680   0.97681   0.97669      1035
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98910   0.98776   0.98843       735
        spam    0.97010   0.97333   0.97171       300

    accuracy                        0.98357      1035
   macro avg    0.97960   0.98054   0.98007      1035
weighted avg 

In [66]:
for i in range(6):
    cv = tx.CountVectorizer()
    cv.fit(corpus['text'])
    tr_xtrain = cv.transform(xtrain)
    tr_xtest = cv.transform(xtest)
    rfc = en.RandomForestClassifier()
    rfc.fit(tr_xtrain,ytrain)

    imp = pd.DataFrame({'name' : cv.vocabulary_.keys(), 'importance' : rfc.feature_importances_}) 
    imp = imp.sort_values(by = 'importance', ascending = False)
    top100 = imp.head(n=10**i)['name'].to_numpy()

    cv = tx.CountVectorizer(vocabulary = top100)
    cv.fit(corpus['text'])
    tr_xtrain = cv.transform(xtrain)
    tr_xtest = cv.transform(xtest)

    lr =  lm.LogisticRegression()
    rfc = en.RandomForestClassifier()
    cnb = nb.ComplementNB()

    lr.fit(tr_xtrain,ytrain)
    rfc.fit(tr_xtrain,ytrain)
    cnb.fit(tr_xtrain,ytrain)

    pred_lr = lr.predict(tr_xtest)
    pred_rfc = rfc.predict(tr_xtest)
    pred_cnb = cnb.predict(tr_xtest)

    print('количество слов:', len(top100), 'score:', cnb.score(tr_xtest,ytest))
    print(me.classification_report(ytest,pred_cnb,digits=5))

количество слов: 1 score: 0.7101449275362319
              precision    recall  f1-score   support

         ham    0.71014   1.00000   0.83051       735
        spam    0.00000   0.00000   0.00000       300

    accuracy                        0.71014      1035
   macro avg    0.35507   0.50000   0.41525      1035
weighted avg    0.50431   0.71014   0.58978      1035

количество слов: 10 score: 0.7101449275362319
              precision    recall  f1-score   support

         ham    0.71014   1.00000   0.83051       735
        spam    0.00000   0.00000   0.00000       300

    accuracy                        0.71014      1035
   macro avg    0.35507   0.50000   0.41525      1035
weighted avg    0.50431   0.71014   0.58978      1035

количество слов: 100 score: 0.5690821256038647
              precision    recall  f1-score   support

         ham    1.00000   0.39320   0.56445       735
        spam    0.40214   1.00000   0.57361       300

    accuracy                        0.56908 